In [52]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import tensorflow as tf

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

In [25]:
num_classes = 10
epochs = 20

In [26]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [68]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [67]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    # print(y_pred)
    # square_pred = K.square(y_pred)
    square_pred = tf.math.square(y_pred)
    # margin_square = K.square(K.maximum(margin - y_pred, 0))
    margin_square = tf.math.square(tf.maximum(margin - y_pred, 0))
    # return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    # print(y_true)
    # print(y_pred)
    # print(square_pred)
    # print(margin_square)
    return tf.math.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [62]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1., 0.]
    return np.array(pairs), np.array(labels)


In [30]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [31]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

In [32]:
x_train.shape

(60000, 28, 28)

In [63]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

In [34]:
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

In [35]:
print(tr_pairs.shape)
print(tr_y.shape)

(108400, 2, 28, 28)
(108400,)


In [64]:
digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

In [36]:
[len(digit_indices[d]) for d in range(num_classes)]

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]

In [37]:
len(tr_pairs)

108400

In [38]:
# [[A 28X28],[P 28X28]]*54200,[[A],[N]]*54200 | 54200 = 5421-1 | 5421 = min([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949])

In [65]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)


# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)


In [66]:

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [70]:
model.summary()

Model: &quot;functional_13&quot;
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
functional_11 (Functional)      (None, 128)          133504      input_8[0][0]                    
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1)            0           functional_

In [58]:
print(processed_a)

Tensor(&quot;functional_5/dense_5/Relu:0&quot;, shape=(None, 128), dtype=float32)


In [69]:
# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

Epoch 1/20
217/847 [======&gt;.......................] - ETA: 13s - loss: 0.1617 - accuracy: 0.7754

KeyboardInterrupt: 

In [46]:
K.mean(np.array([1,2]))

AttributeError: &#39;numpy.dtype&#39; object has no attribute &#39;base_dtype&#39;